In [5]:
!pip install torch torchvision einops tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from einops import rearrange
from tqdm import tqdm
import numpy as np
import random

In [3]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.1),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=2)


class PatchEmbedding(nn.Module):
    def __init__(self, in_channels=3, patch_size=4, emb_size=256, img_size=32):
        super().__init__()
        self.patch_size = patch_size
        self.n_patches = (img_size // patch_size) ** 2
        self.proj = nn.Conv2d(in_channels, emb_size, kernel_size=patch_size, stride=patch_size)

    def forward(self, x):
        x = self.proj(x)
        x = x.flatten(2)
        x = x.transpose(1,2)
        return x

class Attention(nn.Module):
    def __init__(self, emb_size=256, num_heads=8, dropout=0.1):
        super().__init__()
        self.mha = nn.MultiheadAttention(embed_dim=emb_size, num_heads=num_heads, dropout=dropout, batch_first=True)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out, _ = self.mha(x,x,x)
        return self.dropout(out)

class TransformerBlock(nn.Module):
    def __init__(self, emb_size=256, num_heads=8, mlp_ratio=4., dropout=0.1):
        super().__init__()
        self.norm1 = nn.LayerNorm(emb_size)
        self.norm2 = nn.LayerNorm(emb_size)
        self.attn = Attention(emb_size, num_heads, dropout)
        self.mlp = nn.Sequential(
            nn.Linear(emb_size, int(emb_size*mlp_ratio)),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(int(emb_size*mlp_ratio), emb_size),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        x = x + self.attn(self.norm1(x))
        x = x + self.mlp(self.norm2(x))
        return x

class ViT(nn.Module):
    def __init__(self, img_size=32, patch_size=4, in_channels=3, num_classes=10,
                 emb_size=256, depth=8, num_heads=8, mlp_ratio=4., dropout=0.1):
        super().__init__()
        self.patch_embed = PatchEmbedding(in_channels, patch_size, emb_size, img_size)
        self.cls_token = nn.Parameter(torch.randn(1,1,emb_size))
        self.pos_embed = nn.Parameter(torch.randn(1, (img_size//patch_size)**2 + 1, emb_size))
        self.blocks = nn.ModuleList([TransformerBlock(emb_size, num_heads, mlp_ratio, dropout) for _ in range(depth)])
        self.norm = nn.LayerNorm(emb_size)
        self.head = nn.Linear(emb_size, num_classes)

    def forward(self, x):
        x = self.patch_embed(x)
        B = x.shape[0]
        cls_tokens = self.cls_token.expand(B, -1, -1)
        x = torch.cat([cls_tokens, x], dim=1)
        x = x + self.pos_embed
        for blk in self.blocks:
            x = blk(x)
        cls_out = self.norm(x[:,0])
        out = self.head(cls_out)
        return out

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = ViT().to(device)

criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = optim.AdamW(model.parameters(), lr=3e-4, weight_decay=0.05)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100)

epochs = 100
best_acc = 0.0


for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    loop = tqdm(train_loader, leave=False)
    for imgs, labels in loop:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        preds = model(imgs)
        loss = criterion(preds, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        loop.set_description(f"Epoch [{epoch+1}/{epochs}] Loss: {loss.item():.4f}")

    scheduler.step()
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for imgs, labels in test_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            preds = model(imgs)
            correct += (preds.argmax(1) == labels).sum().item()
            total += labels.size(0)
    acc = correct/total

    if acc > best_acc:
        best_acc = acc
        torch.save(model.state_dict(), "vit_cifar10.pth")

    print(f"Epoch {epoch+1}/{epochs} | Test Acc: {acc:.4f} | Best: {best_acc:.4f}")

print("Accuracy:", best_acc)

Epoch 1/100 | Test Acc: 0.4216 | Best: 0.4216


Epoch 2/100 | Test Acc: 0.4755 | Best: 0.4755


Epoch 3/100 | Test Acc: 0.5022 | Best: 0.5022


Epoch 4/100 | Test Acc: 0.5361 | Best: 0.5361


Epoch 5/100 | Test Acc: 0.5709 | Best: 0.5709


...

Epoch 96/100 | Test Acc: 0.8078 | Best: 0.8094


Epoch 97/100 | Test Acc: 0.8077 | Best: 0.8094


Epoch 98/100 | Test Acc: 0.8069 | Best: 0.8094


Epoch 99/100 | Test Acc: 0.8067 | Best: 0.8094


Epoch 100/100 | Test Acc: 0.8068 | Best: 0.8094
Accuracy: 0.8094
